In [ ]:
## VGG16, feature vector 뽑은 후 거리 계산  (lookbook -cloth_table 내에서..)

from PIL import Image
import os
import six.moves.cPickle as Pickle

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import time

## 이미지 불러오기
with open('cloth_table.pkl', 'rb') as cloth_table:
    cloth_data = Pickle.load(cloth_table)
# cloth_data
print(len(cloth_data)) # 8726


start = time.time()


## VGG16
features = []
vgg_model = VGG16(weights='imagenet')
model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('fc2').output) 

for i in range(len(cloth_data)):
    img_path = '../data/' + cloth_data[i]

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    feature = model.predict(x)
    # print(feature.shape)  # (1,4096)
    features.append(feature)
    
    if i % 50 == 0:
        print(i)

feature_4096 = np.squeeze(np.array(features))


## Euclidean distance
dist = euclidean_distances(feature_4096, feature_4096)


## 계산시간
end = time.time() - start
print(end)  # 12955.27237534523


## 결과 저장
with open('vgg16_features_4096.pkl', 'wb') as vgg16_features_4096:
    Pickle.dump(feature_4096, vgg16_features_4096)
    
with open('vgg16_features_4096_dist.pkl', 'wb') as vgg16_features_4096_dist:
    Pickle.dump(dist, vgg16_features_4096_dist)

In [ ]:
## 특정 이미지와 유사한 상위 5개 이미지 추출  (lookbook -cloth_table 내에서..)

from PIL import Image
import os
import six.moves.cPickle as Pickle
import numpy as np

## 이미지 불러오기
with open('cloth_table.pkl', 'rb') as cloth_table:
    cloth_data = Pickle.load(cloth_table)

## 결과 불러오기(feature vector)
with open('vgg16_features_4096.pkl', 'rb') as vgg16_features_4096:
    vgg16_features_4096 = Pickle.load(vgg16_features_4096)
# vgg16_features_4096.shape  # (8726, 4096)
vgg16_features_4096

## 결과 불러오기(euclidean distance)
with open('vgg16_features_4096_dist.pkl', 'rb') as vgg16_features_4096_dist:
    vgg16_features_4096_dist = Pickle.load(vgg16_features_4096_dist)
# vgg16_features_4096_dist.shape  # (8726, 8726)
vgg16_features_4096_dist


## 특정 이미지 선택 후 유사 이미지 보기(top5)
i = 4  # i번째 이미지 선택
top = 5

top_ind = np.argsort(vgg16_features_4096_dist[i,])[:(top+1)]

j = 0
for ind in top_ind:     
    img_path = '../data/' + cloth_data[ind]   
    globals()['top{}'.format(j)] = Image.open(img_path)
    j += 1 

top0  # 선택 이미지(=i번째 이미지)
# top1  # 선택 이미지와 가장 유사한 이미지 top 1 
# top2  # 선택 이미지와 가장 유사한 이미지 top 2
# top3
# top4
# top5


In [ ]:
## 새로운 이미지 추가시 유사 이미지 찾기

# find_images_VGG16(img_name, top_num)
# img_name: 새로운 이미지 파일명
# top_num: 0=자기자신, 1=가장 유사한 이미지 top1, 2=가장유사한 이미지 top2.

def find_images_VGG16(img_name, top_num):
    from PIL import Image
    import os
    import six.moves.cPickle as Pickle

    from keras.applications.vgg16 import VGG16
    from keras.preprocessing import image
    from keras.applications.vgg16 import preprocess_input
    from keras.models import Model
    
    import numpy as np
    from sklearn.metrics.pairwise import euclidean_distances

    ## VGG16
    vgg_model = VGG16(weights='imagenet')
    model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('fc2').output) 

    img_path = '../ex/' + img_name + '.png'  # 새로운 이미지 저장된 경로 지정

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    ex_feature = model.predict(x)


    ## 기존 이미지(lookbook -cloth_table)의 feature vector 불러오기   
    with open('vgg16_features_4096.pkl', 'rb') as vgg16_features_4096:
        vgg16_features_4096 = Pickle.load(vgg16_features_4096)


    ## 이미지간 거리 계산(euclidean distance)
    dist = euclidean_distances(ex_feature, vgg16_features_4096)
    dist = np.squeeze(dist)


    ## 기존 이미지 불러오기
    with open('cloth_table.pkl', 'rb') as cloth_table:
        cloth_data = Pickle.load(cloth_table)

        
    ## 유사 이미지 선택   
    top_ind = np.argsort(dist)

    if (top_num == 0):
        result = Image.open(img_path)  
    if (top_num > 0):
        result = Image.open('../data/' + cloth_data[top_ind[top_num-1]]) 
    return result


## 함수 실행
# find_images_VGG16("ex2",0)  # 내가 넣은 새로운 이미지 출력
find_images_VGG16("ex2",1)  # 새로운 이미지와 가장 유사한 이미지 top1
# find_images_VGG16("ex2",2)  # 새로운 이미지와 가장 유사한 이미지 top2 
# find_images_VGG16("ex2",3)  
# find_images_VGG16("ex2",4)  
# find_images_VGG16("ex2",5)
